In [2]:
import numpy as np
from transformers import BertTokenizer, TFBertModel

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)
texts = ["這是一個很棒的產品！", "這家餐廳的服務很差...", "電影的劇情令人感動！"]
inputs = tokenizer(
    texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
outputs = bert_model(**inputs)
cls_embeddings = outputs.last_hidden_state[:, 0, :]
print("BERT 特徵維度:", cls_embeddings.shape)
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(768,))  # BERT 特徵維度
x = Dense(128, activation="relu")(input_layer)
x = Dense(64, activation="relu")(x)
output_layer = Dense(2, activation="softmax")(x)  # 二分類問題

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

I0000 00:00:1743307611.945533   54775 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model 

BERT 特徵維度: (3, 768)


In [3]:
classifier = Model(inputs=input_layer, outputs=output_layer)
classifier.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
X_train = np.random.rand(100, 768)  # 100 筆訓練樣本，每個 768 維
y_train = np.random.randint(0, 2, 100)  # 0（負面）或 1（正面）
classifier.fit(X_train, y_train, epochs=5, batch_size=8)
test_texts = ["這部電影真的很棒！", "這間餐廳讓我失望透頂...", "這款手機的性能很不錯"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]  # 取 CLS 向量
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)
# 假設 1 代表正向，0 代表負向

Epoch 1/5


I0000 00:00:1743307618.206774   55816 service.cc:152] XLA service 0x7f75d800e4a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743307618.206818   55816 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-30 12:06:58.229983: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743307618.340942   55816 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/13 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5000 - loss: 0.7696

I0000 00:00:1743307619.713507   55816 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.5436 - loss: 0.7395
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6718 - loss: 0.6053 
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7877 - loss: 0.5093 
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8444 - loss: 0.4115 
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9537 - loss: 0.3237 
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 806ms/step
預測結果: [1 1 1]


In [4]:
test_texts = ["這部電影不錯，但是有點無聊。", "這部電影有點無聊，但是不錯。"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)
test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 811ms/step
預測結果: [1 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
預測結果: [1]
